# Deploy Code Patch in Bulk
This script deploys updates via a git patch to multiple repositories that were created from one code template. 
The code does the following:
* Clones all repositories to the hard drive
* Checks out the develop branch on each repository
* Applies a git patch to the code
* Commit the code on the develop branch
* Deploy the code to the develop branch for testing
* Once the code is approves, it deploys a git flow release to the master branch

## Prerequisites
* Python 3 and VSCode to run this notebook
* git and git flow
* Each repository contains package.json with a version number in it
* Each repository contains changelog.md with `## Unreleased` text at the top

## Dependencies

In [ ]:
import datetime
import json
import os
import re
import subprocess

## Variables

In [ ]:
# Define variables, for reuse
parent_dir = '/home/projects'
# Flag so the code know whether to run dev/update or release scripts
is_release = False

patch_name = 'code-patch.diff'
patch_location = parent_dir + patch_name

# List all repos as a list for ease of human reading and updating
all_projects = {
  'ca': {
    'path_project': parent_dir + '/template/ca/',
    'repository': 'git@bitbucket.org:organization/ca-from-template.git',
    'url': 'https://example.com/ca-test-url',
  },
  'ct': {
    'path_project': parent_dir + '/template/ct/',
    'repository': 'git@bitbucket.org:organization/ct-from-template.git',
    'url': 'https://example.com/ct-test-url',
  },
  'ma': {
    'path_project': parent_dir + '/template/ma/',
    'repository': 'git@bitbucket.org:organization/ma-from-template..git',
    'url': 'https://example.com/ma-test-url',
  },
  'sd': {
    'path_project': parent_dir + '/template/sd/',
    'repository': 'git@bitbucket.org:organization/sd-from-template..git',
    'url': 'https://example.com/sd-test-url',
  }, 
}

## Helper function

In [ ]:
# Searches and replaces pattern in a file
def apply_search_replace(path, search_replace, write = True):
  if os.path.exists(path):
    with open(path, 'r+') as file:
      data = file.read()
      data_new = data

      for item in search_replace:
          data_new = re.sub(item['search'], item['replace'], data_new)

      if (write):
        file.seek(0)
        file.write(data_new)
        file.truncate()
      print(f'updated {path}')
  else:
    raise ValueError(f'not found {path}')

# Reads the version from package.json 
def get_current_version(path):
  version = '0.1.0'
  if os.path.exists(path):
    with open(path, 'r') as file:
      package = json.loads(file.read())
      version = package['version']
  else:
    raise ValueError(f'not found {path}')
  return version


# Increments version by 0.1.0
def increase_version(version, increment = '0.1.0'):
  version_list = version.split('.')
  increment_list = increment.split('.')
  new_version_list = []
  for i in range(len(version_list)):
    if (i == 2 and increment == '0.1.0'):
      new_version_list.append(0)
    else:
      new_version_list.append(int(version_list[i]) + int(increment_list[i]))

  new_version = '.'.join(map(str, new_version_list))
  return new_version

# Updates the changelog file with info about a new feature
def update_changelog_feature(path, write=True):
  change_txt = '- Added new feature X'
  search_replace = [
    {
      'search': r'(.*)(## \[Unreleased\]([\s\n\r]*))',
      'replace': r'\g<1>\g<2>### Changed\n' + change_txt + '\n\n',
    },
  ]

  apply_search_replace(path, search_replace, write)

# Updates the changelog file with a new release
def update_changelog_release(path, version, write = True):
  today = datetime.datetime.now().strftime("%Y-%m-%d")
  release_title = f"[{version}] {today}"
  search_replace = [
    {
      'search': r'(.*)(## \[Unreleased\]([\s\n\r]*))',
      'replace': '\g<1>\g<2>\n## ' + release_title + ' \n\n',
    },
  ]

  apply_search_replace(path, search_replace, write)

# Update package file for release
def update_package_release(path, version, write = True):
  search_replace = [
    {
      'search': r'"version": ".*"', 
      'replace': f'"version": "{version}"',
    }
  ]
  apply_search_replace(path, search_replace, write)


## Top level functions

In [ ]:
def get_repositories(all_projects):
  for key, item in all_projects.items():
    if not os.path.exists(item['path_project']):
      path_project = item['path_project'] 
      repository = item['repository']
      command_clone = f'git clone {repository} {path_project}'
      subprocess.run(command_clone, shell=True)

def release_code(all_projects, write=True):
  command_add = 'git add -u'
  command_pull_master = 'git pull origin master'
  command_push_develop = 'git push origin develop'
  command_push_master = 'git push origin master --follow-tags'
  command_release_start = 'git flow release start '
  command_release_finish = 'git flow release finish -m '
  command_release_commit = 'git commit -m "Version increase for release"'
  path_changelog = item['path_project'] + 'changelog.md'
  path_package = item['path_project'] + 'package.json'

  for key, item in all_projects.items():
    version = get_current_version(path_package)
    version = increase_version(version)
    subprocess.run(command_release_start + version, shell=True)
    update_changelog_release(path_changelog, version, write)
    update_package_release(path_package, version, write)
    subprocess.run(command_add, shell=True)
    subprocess.run(command_release_commit, shell=True)
    subprocess.run(command_release_finish + f"'{version}' {version}", shell=True)
    subprocess.run(command_pull_master, shell=True)
    subprocess.run(command_push_master, shell=True)
    subprocess.run(command_push_develop, shell=True)

# Main function for code execution
def update_code(all_projects, patch_name, patch_location, write=True):
  command_checkout = 'git checkout develop'
  command_pull_develop = 'git pull origin develop'
  command_reset = 'git checkout develop -f'
  # note this will commit everyting, use git add -u when the intention is to commit only updated files
  command_add = 'git add *' 
  command_commit = 'git commit -m "Added new feature X"'
  command_push_develop = 'git push origin develop'
  
  for key, item in all_projects.items():
    path_project = item['path_project'] 
    path_changelog = path_project + item['file_changelog'] 
    command_copy_patch = f'cp {patch_location} ./'
    command_apply_patch = f'git apply {patch_location}' 
    command_remove_patch = f'rm {patch_name}'
  
    # This condition can be used for testing purposes when testing and developing the code, 
    # Disable for the final run, included for ease of toggling on/off
    # if key == 'ca':
    os.chdir(path_project)
    subprocess.run(command_checkout, shell=True)
    # reset so if we run the code repeatedly we don't create multiple replacements
    # helpful when perfecting the code
    subprocess.run(command_reset, shell=True)
    subprocess.run(command_pull_develop, shell=True)
    update_changelog_feature(path_changelog, write)
    subprocess.run(command_copy_patch, shell=True)
    subprocess.run(command_apply_patch, shell=True)
    subprocess.run(command_remove_patch, shell=True)
    subprocess.run(command_add, shell=True)
    subprocess.run(command_commit, shell=True)
    subprocess.run(command_push_develop, shell=True)


## Run the script

In [ ]:

if not is_release:
  get_repositories(all_projects)
  update_code(all_projects, patch_name, patch_location)
else:
  release_code(all_projects)